In [1]:
import pandas as pd
import json
import numpy as np

# Transformação dos dados TJ - Educação

## Carregar os dados

In [2]:
try:
    df = pd.read_csv("../data/data.csv")
except:
    years = [year for year in range(2015, 2024)]

    df = pd.DataFrame()
    for year in years:
        # Lê o arquivo csv correspondente ao ano
        aux = pd.read_csv(f'../data/{year}.csv', encoding='latin-1', sep=';')
        aux['ANO'] = year

        df = pd.concat([df, aux], ignore_index=True,).reset_index(drop=True)

    df.to_csv("../data/data.csv", index=False)
print(df.shape)
df.head()

(30944, 139)


,NO_MUNICIPIO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_POTAVEL,...,Unnamed: 129,ANO,IN_PROF_GESTAO,IN_ACESSIBILIDADE_ELEVADOR,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR,IN_ALMOXARIFADO,IN_AREA_VERDE,IN_AUDITORIO,QT_MAT_FUND_INT
0,Almadina,3.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Almadina,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Almadina,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Almadina,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Almadina,3.0,0.0,2.0,0.0,1.0,3.0,0.0,1.0,0.0,...,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Processamento nos dados do DF

In [3]:
# Função para converter float para int
def float_to_int(value):
    try:
        return int(value)
    except:
        return "NaN"

# Converte as colunas para inteiros
for column in df.columns:
    if column != 'NO_MUNICIPIO':
        df[column] = df[column].apply(float_to_int)

    if 'Unnamed' in column:
        del df[column]

columns = df.columns
columns = [column for column in columns if column not in ['NO_MUNICIPIO', 'ANO']]
df = df[['NO_MUNICIPIO', 'ANO'] + columns]

# Preenche os valores NaN
df.fillna('NaN', inplace=True)

# Carrega o dicionário de categorias
dict_tp = json.load(open('../output/dict_categoria.json', 'r'))

# Loop para mapear as categorias
for key in dict_tp:
    # Verifica se a coluna existe no dataframe
    if key in df.columns:
        tmp = {}

        # Mapeia as categorias
        if 'categoria' in dict_tp[key]:
            for k, v in dict_tp[key]['categoria'].items():
                # Verifica se o valor é NaN
                if k == 'NaN':
                    tmp[k] = v

                    if '0' not in tmp.keys() and 0 not in tmp.keys():
                        tmp[0] = v
                else:
                    tmp['NaN'] = "N/A"
                    tmp[int(k)] = v

            df[key] = df[key].map(tmp)

for column in df.columns:
    if column not in dict_tp.keys() and 'IN' in column:
        df[column] = df[column].map({0: 'Não', 1: 'Sim', 'NaN': 'N/A'})

df.replace('NaN', "N/A", inplace=True)

df.head()

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_ACESSIBILIDADE_ELEVADOR,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR,IN_ALMOXARIFADO,IN_AREA_VERDE,IN_AUDITORIO,QT_MAT_FUND_INT
0,Almadina,2015,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Próprio,Não,Sim,...,0,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Sim,Cedido,Não,Sim,...,0,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [4]:
try:
    with open('../data/cities.txt', 'r') as fd:
        cities = fd.readlines()
        cities = [city.strip().replace("\n", "") for city in cities]

    print("Before", df.shape)
    df = df[df['NO_MUNICIPIO'].isin(cities)]
    print("After", df.shape)
except FileNotFoundError:
    print("Arquivo não encontrado")


Before (30944, 119)
After (10810, 119)


## Agrupar e converter dados para JSON

In [5]:
tmp = df[['ANO']].copy()
tmp =  tmp.rename(columns={'ANO': 'count'})
df = pd.concat([df, tmp], axis=1)

out_json = {}

columns = df.columns
columns = [column for column in columns if column not in ['NO_MUNICIPIO', 'ANO', 'count']]

for column in columns:
    groupby_df = df.groupby(['NO_MUNICIPIO', 'ANO', column],
                            as_index=False).agg({'count': 'count'})
    tmp = groupby_df.to_dict(orient='split')

    for i in range(len(tmp['data'])):
        municipio = tmp['data'][i][0]
        ano = tmp['data'][i][1]
        categoria = tmp['data'][i][2]
        count = tmp['data'][i][3]

        if municipio not in out_json:
            out_json[municipio] = {}

        if ano not in out_json[municipio]:
            out_json[municipio][ano] = {}

        if column not in out_json[municipio][ano]:
            out_json[municipio][ano][column] = {}

        out_json[municipio][ano][column][categoria] = count

json.dump(out_json, open('../output/data.json', 'w'), ensure_ascii=False)

In [6]:
out_json['Ibicaraí']

{2015: {'TP_DEPENDENCIA': {'Estadual': 6, 'Municipal': 23, 'Privada': 8},
  'TP_CATEGORIA_ESCOLA_PRIVADA': {'Filantrópica': 1,
   'Não aplicável para escolas públicas': 29,
   'Particular': 7},
  'TP_LOCALIZACAO': {'Rural': 13, 'Urbana': 24},
  'TP_LOCALIZACAO_DIFERENCIADA': {'Escola não está em área de localização diferenciada': 37},
  'TP_REGULAMENTACAO': {'Em tramitação': 2, 'Não': 3, 'Sim': 32},
  'TP_OCUPACAO_PREDIO_ESCOLAR': {'Alugado': 3,
   'Cedido': 9,
   'Não aplicável para escolas que não ocupam prédio escolar': 8,
   'Próprio': 17},
  'IN_PREDIO_COMPARTILHADO': {'Não': 27, 'Sim': 10},
  'IN_AGUA_FILTRADA': {'Não': 2, 'Sim': 35},
  'IN_AGUA_POTAVEL': {'Não': 37},
  'IN_AGUA_REDE_PUBLICA': {'Não': 4, 'Sim': 33},
  'IN_AGUA_POCO_ARTESIANO': {'Não': 36, 'Sim': 1},
  'IN_AGUA_CACIMBA': {'Não': 37},
  'IN_AGUA_FONTE_RIO': {'Não': 35, 'Sim': 2},
  'IN_AGUA_INEXISTENTE': {'Não': 37},
  'IN_ENERGIA_REDE_PUBLICA': {'Não': 2, 'Sim': 35},
  'IN_ENERGIA_GERADOR': {'Não': 37},
  'IN_ENER